## Data Engineering First Project

### importing libraries

In [1]:
import psycopg2
import pandas as pd

### Create database function

In [2]:
def create_database():
    conn = psycopg2.connect("host=127.0.0.1 dbname = postgres user = postgres password = postgres")
    conn.set_session(autocommit = True)
    cur = conn.cursor()
    
    # create a accounts database
    cur.execute("DROP DATABASE accounts")
    cur.execute("CREATE DATABASE accounts")
    
    # close the connection to the default database
    conn.close()
    
    # connect to the accounts database
    conn = psycopg2.connect("host=127.0.0.1 dbname = accounts user = postgres password = postgres")
    cur = conn.cursor()
    
    return conn, cur

### create a function to drop the tables

In [3]:
def drop_tables(cur,conn):
    for query in drop_table_queries:
        cur.execute(query)
        conn.commit()

### function to create a tables

In [4]:
def create_tables(cur,conn):
    for query in create_table_queries:
        cur.execute(query)
        conn.commit()

### Reading the Wealth-AccountsCountry.csv file

In [5]:
Accounts_country = pd.read_csv("data/Wealth-AccountsCountry.csv")

In [6]:
Accounts_country.head()

,Code,Long Name,Income Group,Region,Lending category,Other groups,Currency Unit,Latest population census,Latest household survey,Special Notes,...,Source of most recent Income and expenditure data,Vital registration complete,Latest agricultural census,Latest industrial data,Latest trade data,Latest water withdrawal data,2-alpha code,WB-2 code,Table Name,Short Name
0,ALB,Republic of Albania,Upper middle income,Europe & Central Asia,IBRD,NaN,Albanian lek,2020 (expected),"Demographic and Health Survey, 2017/18",NaN,...,Living Standards Measurement Study Survey (LSM...,Yes,2012,2013.0,2018.0,2006.0,AL,AL,Albania,Albania
1,ARG,Argentine Republic,Upper middle income,Latin America & Caribbean,IBRD,NaN,Argentine peso,2020 (expected),"Multiple Indicator Cluster Survey, 2019/20",NaN,...,"Integrated household survey (IHS), 2016",Yes,2008,2002.0,2018.0,2011.0,AR,AR,Argentina,Argentina
2,ARM,Republic of Armenia,Upper middle income,Europe & Central Asia,IBRD,NaN,Armenian dram,2020 (expected),"Demographic and Health Survey, 2015/16",NaN,...,"Integrated household survey (IHS), 2016",Yes,2014,NaN,2018.0,2012.0,AM,AM,Armenia,Armenia
3,AUS,Commonwealth of Australia,High income,East Asia & Pacific,NaN,NaN,Australian dollar,2016,NaN,Fiscal year end: June 30; reporting period for...,...,"Expenditure survey/budget survey (ES/BS), 2010",Yes,2015-2016,2013.0,2018.0,2013.0,AU,AU,Australia,Australia
4,AUT,Republic of Austria,High income,Europe & Central Asia,NaN,Euro area,Euro,2011. Population figures compiled from adminis...,NaN,A simple multiplier is used to convert the nat...,...,"Income survey (IS), 2015",Yes,2010,2014.0,2018.0,2010.0,AT,AT,Austria,Austria


### choosing few columns as required

In [7]:
Accounts_country_clean = Accounts_country[['Code', 'Short Name', 'Table Name', 'Long Name', 'Currency Unit']]

In [8]:
Accounts_country_clean.head()

,Code,Short Name,Table Name,Long Name,Currency Unit
0,ALB,Albania,Albania,Republic of Albania,Albanian lek
1,ARG,Argentina,Argentina,Argentine Republic,Argentine peso
2,ARM,Armenia,Armenia,Republic of Armenia,Armenian dram
3,AUS,Australia,Australia,Commonwealth of Australia,Australian dollar
4,AUT,Austria,Austria,Republic of Austria,Euro


### Reading Wealth-AccountData.csv file

In [9]:
Account_Data = pd.read_csv("data/Wealth-AccountData.csv")

In [10]:
Account_Data.head()

,Country Name,Country Code,Series Name,Series Code,1995 [YR1995],1996 [YR1996],1997 [YR1997],1998 [YR1998],1999 [YR1999],2000 [YR2000],...,2009 [YR2009],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018]
0,Albania,ALB,Human capital (constant 2018 US$),NW.HCA.TO,44900000000.00,43400000000.00,37100000000.00,38800000000.00,42200000000.00,43600000000.00,...,6.610000e+10,6.810000e+10,6.850000e+10,7.080000e+10,7.160000e+10,7.250000e+10,7.370000e+10,75700000000.00,78000000000.00,81200000000.00
1,Albania,ALB,Human capital per capita (constant 2018 US$),NW.HCA.PC,14072.79,13708.99,11794.10,12399.48,13573.26,14118.13,...,2.258200e+04,2.336286e+04,2.359058e+04,2.439397e+04,2.473162e+04,2.509002e+04,2.559371e+04,26326.26,27160.86,28333.68
2,Albania,ALB,"Human capital per capita, employed female (con...",NW.HCA.FEMP.PC,3468.86,3355.55,2869.00,3013.39,3259.89,3424.01,...,2.825340e+03,2.889150e+03,2.848520e+03,2.930230e+03,2.954270e+03,2.957420e+03,2.987550e+03,3072.50,3178.42,3312.93
3,Albania,ALB,"Human capital per capita, employed male (const...",NW.HCA.MEMP.PC,6507.93,6353.74,5466.69,5901.07,6492.73,6872.02,...,1.525319e+04,1.578992e+04,1.579854e+04,1.647360e+04,1.681013e+04,1.699138e+04,1.730701e+04,17817.10,18445.94,19245.19
4,Albania,ALB,"Human capital per capita, female (constant 201...",NW.HCA.FEMA.PC,4893.01,4737.86,4059.32,4191.45,4536.97,4695.07,...,3.529140e+03,3.613610e+03,3.603690e+03,3.683810e+03,3.696740e+03,3.719610e+03,3.767630e+03,3872.14,3992.19,4161.14


### printing the column name

In [11]:
Account_Data.columns

Index(['Country Name', 'Country Code', 'Series Name', 'Series Code',
       '1995 [YR1995]', '1996 [YR1996]', '1997 [YR1997]', '1998 [YR1998]',
       '1999 [YR1999]', '2000 [YR2000]', '2001 [YR2001]', '2002 [YR2002]',
       '2003 [YR2003]', '2004 [YR2004]', '2005 [YR2005]', '2006 [YR2006]',
       '2007 [YR2007]', '2008 [YR2008]', '2009 [YR2009]', '2010 [YR2010]',
       '2011 [YR2011]', '2012 [YR2012]', '2013 [YR2013]', '2014 [YR2014]',
       '2015 [YR2015]', '2016 [YR2016]', '2017 [YR2017]', '2018 [YR2018]'],
      dtype='object')

In [12]:
AccountsData = Account_Data[['Country Name', 'Country Code', 'Series Name','Series Code', '1995 [YR1995]','2000 [YR2000]','2005 [YR2005]','2010 [YR2010]','2015 [YR2015]']]
AccountsData.head()

,Country Name,Country Code,Series Name,Series Code,1995 [YR1995],2000 [YR2000],2005 [YR2005],2010 [YR2010],2015 [YR2015]
0,Albania,ALB,Human capital (constant 2018 US$),NW.HCA.TO,44900000000.00,43600000000.00,5.440000e+10,6.810000e+10,7.370000e+10
1,Albania,ALB,Human capital per capita (constant 2018 US$),NW.HCA.PC,14072.79,14118.13,1.807230e+04,2.336286e+04,2.559371e+04
2,Albania,ALB,"Human capital per capita, employed female (con...",NW.HCA.FEMP.PC,3468.86,3424.01,2.222120e+03,2.889150e+03,2.987550e+03
3,Albania,ALB,"Human capital per capita, employed male (const...",NW.HCA.MEMP.PC,6507.93,6872.02,1.168935e+04,1.578992e+04,1.730701e+04
4,Albania,ALB,"Human capital per capita, female (constant 201...",NW.HCA.FEMA.PC,4893.01,4695.07,2.886740e+03,3.613610e+03,3.767630e+03


In [13]:
import pandas as pd

# replace all occurrences of ".." with NaN
AccountsData = AccountsData.replace("..", pd.np.nan)

# check for any NaN values in the dataframe
print(AccountsData.isna().sum())

Country Name      0
Country Code      0
Series Name       0
Series Code       0
1995 [YR1995]    10
2000 [YR2000]     2
2005 [YR2005]     0
2010 [YR2010]     0
2015 [YR2015]     0
dtype: int64


C:\Users\91954\AppData\Local\Temp/ipykernel_33912/3618610193.py:4: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  AccountsData = AccountsData.replace("..", pd.np.nan)


In [14]:
AccountsData.head()

,Country Name,Country Code,Series Name,Series Code,1995 [YR1995],2000 [YR2000],2005 [YR2005],2010 [YR2010],2015 [YR2015]
0,Albania,ALB,Human capital (constant 2018 US$),NW.HCA.TO,44900000000.00,43600000000.00,5.440000e+10,6.810000e+10,7.370000e+10
1,Albania,ALB,Human capital per capita (constant 2018 US$),NW.HCA.PC,14072.79,14118.13,1.807230e+04,2.336286e+04,2.559371e+04
2,Albania,ALB,"Human capital per capita, employed female (con...",NW.HCA.FEMP.PC,3468.86,3424.01,2.222120e+03,2.889150e+03,2.987550e+03
3,Albania,ALB,"Human capital per capita, employed male (const...",NW.HCA.MEMP.PC,6507.93,6872.02,1.168935e+04,1.578992e+04,1.730701e+04
4,Albania,ALB,"Human capital per capita, female (constant 201...",NW.HCA.FEMA.PC,4893.01,4695.07,2.886740e+03,3.613610e+03,3.767630e+03


### reading Wealth-AccountSeries.csv file

In [15]:
Account_Serices = pd.read_csv("data/Wealth-AccountSeries.csv")

In [16]:
Account_Serices.columns

Index(['Code', 'Indicator Name', 'Long definition', 'Source', 'Topic',
       'Unit of measure', 'Periodicity', 'Reference period',
       'Statistical concept and methodology', 'Previous Indicator Code',
       'Previous Indicator Name'],
      dtype='object')

In [17]:
AccountsSerices = Account_Serices[['Code', 'Topic', 'Indicator Name', 'Long definition']]

In [18]:
AccountsSerices.head()

,Code,Topic,Indicator Name,Long definition
0,NW.HCA.TO,Human capital,Human capital (constant 2018 US$),Human capital is computed as the present value...
1,NW.HCA.PC,Human capital,Human capital per capita (constant 2018 US$),Human capital is computed as the present value...
2,NW.HCA.FEMP.PC,Human capital,"Human capital per capita, employed female (con...",Human capital is computed as the present value...
3,NW.HCA.MEMP.PC,Human capital,"Human capital per capita, employed male (const...",Human capital is computed as the present value...
4,NW.HCA.FEMA.PC,Human capital,"Human capital per capita, female (constant 201...",Human capital is computed as the present value...


### calling create_database function

In [19]:
conn, cur = create_database()

### accountscountry table creation

In [20]:
accounts_country_table_create = ("""CREATE TABLE IF NOT EXISTS accountscountry(
country_code VARCHAR,
short_name VARCHAR,
table_name VARCHAR,
long_name VARCHAR,
currency_unit VARCHAR
)""")

In [21]:
cur.execute(accounts_country_table_create)
conn.commit()

### create accounts_data table

In [22]:
accounts_data_table_create = ("""CREATE TABLE IF NOT EXISTS accountsdata(
country_name VARCHAR,
country_code VARCHAR,
indicator_name VARCHAR,
indicator_code VARCHAR,
year_1995 decimal,
year_2000 decimal,
year_2005 decimal,
year_2010 decimal,
year_2015 decimal
)""")

In [23]:
cur.execute(accounts_data_table_create)
conn.commit()

### create accounts_series table

In [24]:
accountsserices_data_table_create = ("""CREATE TABLE IF NOT EXISTS accountsseries(
series_code VARCHAR,
topic VARCHAR,
indicator_name VARCHAR,
long_definition VARCHAR
)""")

cur.execute(accountsserices_data_table_create)
conn.commit()

### Inserting the data into accountscountry table

In [25]:
accounts_country_table_data_insert = ("""INSERT INTO accountscountry(
country_code,
short_name,
table_name,
long_name,
currency_unit) VALUES (%s,%s,%s,%s,%s)
""");

In [26]:
for i, row in Accounts_country_clean.iterrows():
    #print(list(row))
    cur.execute(accounts_country_table_data_insert, list(row))

In [27]:
conn.commit()

### inserting data into accountsdata table

In [28]:
accounts_data_table_insert = ("""INSERT INTO accountsdata(
country_name,
country_code,
indicator_name,
indicator_code,
year_1995,
year_2000,
year_2005,
year_2010,
year_2015) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)
""");

In [29]:
for i, row in AccountsData.iterrows():
    cur.execute(accounts_data_table_insert, list(row))

In [30]:
conn.commit()

### inserting data into accountsseries table

In [31]:
accounts_series_table_data_insert = ("""INSERT INTO accountsseries(
series_code,
topic,
indicator_name,
long_definition) VALUES (%s,%s,%s,%s)
""");

In [32]:
for i, row in AccountsSerices.iterrows():
    cur.execute(accounts_series_table_data_insert, list(row))

In [33]:
conn.commit()

In [34]:
cur.execute("select * from accountsseries")

In [35]:
cur.close()

In [36]:
conn.close()